# Import Necessary Libiraries

In [ ]:
import os
import numpy as np
from google.colab import drive
import tensorflow as tf

# Check GPU

In [ ]:
%tensorflow_version 2.x
device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')

print('Found GPU at: {}'.format(device_name))

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Found GPU at: /device:GPU:0


# Data Understanding

In [ ]:
# connect with google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
trainData_Dir = "/content/drive/MyDrive/EmoLearn/data/train"
testData_Dir  = "/content/drive/MyDrive/EmoLearn/data/test"

In [ ]:
#os.listdir(trainData_Dir + "/happy")

In [ ]:
'''
import cv2 # reading image (blue color bias)
image = cv2.imread(trainData_Dir+"/happy"+"/Training_99173728.jpg")
import matplotlib.pyplot as plt   #matplotlib is good for map and chart
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image_rgb)
'''

# Data Preprocessing

In [ ]:
batch_size = 64

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
# Initialize image data generator with rescaling
train_datagen = ImageDataGenerator(
    zoom_range = 0.2,
    shear_range = 0.2,
    horizontal_flip = True,
    rescale = 1./255
)
test_datagen = ImageDataGenerator(
    zoom_range = 0.2,
    shear_range = 0.2,
    horizontal_flip = True,
    rescale = 1./255
)

# Preprocess all train images
train_generator = train_datagen.flow_from_directory(
        trainData_Dir,
        target_size=(48, 48),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')

# Preprocess all test images
test_generator = test_datagen.flow_from_directory(
        testData_Dir,
        target_size=(48, 48),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')


Found 28738 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [ ]:
num_of_training_img = 28738
num_of_testing_img = 7178

In [ ]:
train_generator.class_indices

{'angry': 0,
 'disgust': 1,
 'fear': 2,
 'happy': 3,
 'neutral': 4,
 'sad': 5,
 'surprise': 6}

In [ ]:
# label and files count
from collections import Counter
count_train = Counter(train_generator.classes)
print("Train Data : ", count_train.items())
count_test = Counter(test_generator.classes)
print("Test Data : ", count_test.items())

Train Data :  dict_items([(0, 3995), (1, 436), (2, 4097), (3, 7244), (4, 4965), (5, 4830), (6, 3171)])
Test Data :  dict_items([(0, 958), (1, 111), (2, 1024), (3, 1774), (4, 1233), (5, 1247), (6, 831)])


# Modeling

In [ ]:
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

# Data Training & Modelling ANN

In [ ]:
# Artificial Neural Network (ANN)
import tensorflow as tf
'''
ann_model  = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape = (48, 48, 1)),
        tf.keras.layers.Dense(100,activation=tf.nn.relu),
        tf.keras.layers.Dense(500,activation=tf.nn.relu),
        tf.keras.layers.Dense(7,activation=tf.nn.softmax)
])
'''
# Create the model
ann_model = Sequential()
# Add the input layer and the first hidden layer
ann_model.add(Dense(units=64, activation='relu', input_dim=(48, 48, 1)))
# Add more hidden layers
ann_model.add(Dense(units=32, activation='relu'))
ann_model.add(Dense(units=16, activation='relu'))
# Add the output layer
ann_model.add(Dense(units=7, activation='softmax'))

# Compile the model
ann_model.compile(optimizer='adam',
              loss = 'categorical_crossentropy',
              metrics =['accuracy'])

# Train the ANN model
annTrain = ann_model.fit(
        train_generator,
        steps_per_epoch= num_of_training_img // batch_size,
        epochs= 30,
        validation_data=test_generator,
        validation_steps= num_of_testing_img // batch_size)


# Evalutaion for ANN

In [ ]:
h = annTrain.history
h.keys()

In [ ]:
y_predicted = ann_model.predict(test_generator)

In [ ]:
y_pred = []
for i in y_predicted:
  print(i)
  print(type(i))
  print(np.argmax(i))
  y_pred.append(np.argmax(i))

In [ ]:
y_test = test_generator.classes.tolist()
y_test

In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test,y_pred)
print('confusion_matrix')
print(confusion_matrix)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

print('Accuracy Score :',accuracy_score(y_test,y_pred)*100,'%')
print('Precision Macro Score : ',precision_score(y_test,y_pred,average = 'macro')*100,'%')
print('Recall_Score :',recall_score(y_test,y_pred, average = 'macro')*100,'%')
print('F1_Score :',f1_score(y_test,y_pred, average = 'macro')*100,'%')

In [ ]:
# Plotting the training accuracy of ann model
import matplotlib.pyplot as plt

acc = annTrain.history['accuracy']
epochs = range(len(acc))

plt.plot(epochs, acc, label='Accuracy')
plt.title('Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
# Plotting the training loss of ann model
import matplotlib.pyplot as plt

loss = annTrain.history['loss']
epochs = range(len(loss))

plt.plot(epochs, loss, label='Loss')
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

# Data Training & Modelling CNN

In [ ]:
# Create cnn instance of the Sequential model
cnn_Model = Sequential()

# Add convolutional layers
cnn_Model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
cnn_Model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_Model.add(Conv2D(128, (3, 3), activation='relu'))
cnn_Model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_Model.add(Dropout(0.25))

cnn_Model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
cnn_Model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_Model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
cnn_Model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_Model.add(Dropout(0.25))

# Flatten the feature maps
cnn_Model.add(Flatten())

# Add fully connected layers
cnn_Model.add(Dense(1024, activation='relu'))
cnn_Model.add(Dropout(0.5))
cnn_Model.add(Dense(7, activation='softmax'))

# Compile the model
cnn_Model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
emotion_model_info = cnn_Model.fit(train_generator,
                    steps_per_epoch= num_of_training_img // batch_size,
                    epochs=30,
                    verbose = 2,
                    shuffle = False)

In [ ]:
'''
import requests
import time

while True:
    try:
        requests.get('https://www.google.com')
        print("Kept alive.")
    except:
        print("Failed to keep alive.")
    time.sleep(600)
'''

Kept alive.


# Evaluation for CNN

In [ ]:
# Labelling
class_dictionary = train_generator.class_indices
print('Labels dictionary',class_dictionary)

In [ ]:
cnn_Model.summary

In [ ]:
y_predicted = cnn_Model.predict(test_generator)

In [ ]:
y_pred = []
for i in y_predicted:
  print(i)
  print(type(i))
  print(np.argmax(i))
  y_pred.append(np.argmax(i))

In [ ]:
y_true = test_generator.classes.tolist()
y_true

In [ ]:
# Calculate the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_true,y_pred)
print('confusion_matrix')
print(confusion_matrix)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

print('Accuracy Score :',accuracy_score(y_true,y_pred)*100,'%')
print('Precision Macro Score : ',precision_score(y_true,y_pred,average = 'macro')*100,'%')
print('Recall_Score :',recall_score(y_true,y_pred, average = 'macro')*100,'%')
print('F1_Score :',f1_score(y_true,y_pred, average = 'macro')*100,'%')

In [ ]:
# Plotting the training accuracy of cnn model
import matplotlib.pyplot as plt

acc = emotion_model_info.history['accuracy']
epochs = range(len(acc))

plt.plot(epochs, acc, label='Accuracy')
plt.title('Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
# Plotting the training loss of cnn model
import matplotlib.pyplot as plt

loss = emotion_model_info.history['loss']
epochs = range(len(loss))

plt.plot(epochs, loss, label='Loss')
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

# Save Model File

In [ ]:
model.save('/EmoLearn_model.h5')

In [ ]:
loaded_model = tf.keras.models.load_model('/EmoLearn_model.h5')
